In [1]:
import pandas as pd
from database.select.db_select import get_entire_season

In [2]:
season_info, teams_df, games_df = await get_entire_season(2023)

teams_df.head()

,Location,Name,FullName,Division,Conference
0,Buffalo,Bills,Buffalo Bills,AFC East,AFC
1,Miami,Dolphins,Miami Dolphins,AFC East,AFC
2,New York,Jets,New York Jets,AFC East,AFC
3,New England,Patriots,New England Patriots,AFC East,AFC
4,Kansas City,Chiefs,Kansas City Chiefs,AFC West,AFC


In [3]:
games_df.head()

,Id,SeasonId,Week,WeekName,StartTime,AwayTeam,HomeTeam,AwayScore,HomeScore,Overtime
0,856,25,1,Week 1,2023-09-07 20:20:00,Detroit Lions,Kansas City Chiefs,21,20,False
1,857,25,1,Week 1,2023-09-10 13:00:00,Carolina Panthers,Atlanta Falcons,10,24,False
2,858,25,1,Week 1,2023-09-10 13:00:00,Cincinnati Bengals,Cleveland Browns,3,24,False
3,859,25,1,Week 1,2023-09-10 13:00:00,Jacksonville Jaguars,Indianapolis Colts,31,21,False
4,860,25,1,Week 1,2023-09-10 13:00:00,Arizona Cardinals,Washington Commanders,16,20,False


In [4]:
def other_game_info(row: pd.Series, regular_season_games: int) -> pd.Series:
    if row['AwayTeam'] == row['Team']:
        team_score = row['AwayScore']
        opponent_score = row['HomeScore']
    else:
        team_score = row['HomeScore']
        opponent_score = row['AwayScore']
        
    if team_score > opponent_score:
        result: chr = 'W'
    elif team_score < opponent_score:
        result: chr = 'L'
    else:
        result: chr = 'T'
    
    row['Opponent'] = row['HomeTeam'] if row['Team'] == row['AwayTeam'] else row['AwayTeam']
    row['TeamScore'] = team_score
    row['OpponentScore'] = opponent_score
    row['Result'] = result
    row['HomeGame'] = row['HomeTeam'] == row['Team']
    row['PlayoffGame'] = row['Week'] > regular_season_games
    
    return row

def team_games(games_df: pd.DataFrame, team: str, regular_season_games: int) -> pd.DataFrame:
    teams_games_df = games_df[((games_df['HomeTeam'] == team) | (games_df['AwayTeam'] == team))]
    teams_games_df.insert(0, 'Team', team)
    teams_games_df = teams_games_df.apply(other_game_info, regular_season_games=regular_season_games, axis=1).copy()
    
    return teams_games_df
    
team_game_dfs: dict[str, pd.DataFrame] = {team: team_games(games_df, team, season_info.regular_season_week_count) for team in teams_df['FullName'].to_list()}

team_game_dfs['Minnesota Vikings'].head()

,Team,Id,SeasonId,Week,WeekName,StartTime,AwayTeam,HomeTeam,AwayScore,HomeScore,Overtime,Opponent,TeamScore,OpponentScore,Result,HomeGame,PlayoffGame
6,Minnesota Vikings,862,25,1,Week 1,2023-09-10 13:00:00,Tampa Bay Buccaneers,Minnesota Vikings,20,17,False,Tampa Bay Buccaneers,17,20,L,True,False
16,Minnesota Vikings,872,25,2,Week 2,2023-09-14 20:15:00,Minnesota Vikings,Philadelphia Eagles,28,34,False,Philadelphia Eagles,28,34,L,False,False
40,Minnesota Vikings,896,25,3,Week 3,2023-09-24 13:00:00,Los Angeles Chargers,Minnesota Vikings,28,24,False,Los Angeles Chargers,24,28,L,True,False
51,Minnesota Vikings,907,25,4,Week 4,2023-10-01 13:00:00,Minnesota Vikings,Carolina Panthers,21,13,False,Carolina Panthers,21,13,W,False,False
75,Minnesota Vikings,931,25,5,Week 5,2023-10-08 16:25:00,Kansas City Chiefs,Minnesota Vikings,27,20,False,Kansas City Chiefs,20,27,L,True,False


In [7]:
def result_counts(column: pd.Series) -> pd.Series:
    results: dict[chr, int] = column.value_counts().to_dict()
    
    wins: int = results['W'] if 'W' in results else 0
    losses: int = results['L'] if 'L' in results else 0
    ties: int = results['T'] if 'T' in results else 0
    
    return wins, losses, ties

def win_percentage(wins: int, ties: int, num_games: int) -> float:
    if num_games == 0:
        return 0.0
    
    return round((wins + (ties / 2.0)) / num_games, 3)

def record(row: pd.Series, team_game_dfs: dict[str, pd.DataFrame], teams_df: pd.DataFrame) -> pd.Series:
    team_game_df = team_game_dfs[row['FullName']]
    team_game_df = team_game_df[~team_game_df['PlayoffGame']]
    
    wins, losses, ties = result_counts(team_game_df['Result'])
    
    row['W'] = wins
    row['L'] = losses
    row['T'] = ties
    row['Pct'] = win_percentage(wins, ties, len(team_game_df))
    
    conference_teams: list[str] = teams_df[teams_df['Conference'] == row['Conference']]['FullName'].to_list()
    conference_games_df: pd.DataFrame = team_game_df[(team_game_df['Opponent'].isin(conference_teams))].copy()
    
    wins, losses, ties = result_counts(conference_games_df['Result'])
    
    row['Conf'] = f'{wins}-{losses}-{ties}'
    row['PctConf'] = win_percentage(wins, ties, len(conference_games_df))
    
    division_teams: list[str] = teams_df[(teams_df['Division'] == row['Division'])]['FullName'].to_list()
    division_games_df: pd.DataFrame = conference_games_df[(conference_games_df['Opponent'].isin(division_teams))].copy()
    
    wins, losses, ties = result_counts(division_games_df['Result'])
    
    row['Div'] = f'{wins}-{losses}-{ties}'
    row['PctDiv'] = win_percentage(wins, ties, len(division_games_df))
    
    home_games_df: pd.DataFrame = team_game_df[team_game_df['HomeGame']].copy()
    wins, losses, ties = result_counts(home_games_df['Result'])
    row['Home'] = f'{wins}-{losses}-{ties}'
    
    away_games_df: pd.DataFrame = team_game_df[~team_game_df['HomeGame']].copy()
    wins, losses, ties = result_counts(away_games_df['Result'])
    row['Away'] = f'{wins}-{losses}-{ties}'
    
    
    
    return row

teams_df = teams_df.apply(record, team_game_dfs=team_game_dfs, teams_df=teams_df, axis=1)

teams_df.head(32)

,Location,Name,FullName,Division,Conference,W,L,T,Pct,Conf,PctConf,Div,PctDiv,Home,Away
0,Buffalo,Bills,Buffalo Bills,AFC East,AFC,10,6,0,0.625,6-5-0,0.545,3-2-0,0.6,7-2-0,3-4-0
1,Miami,Dolphins,Miami Dolphins,AFC East,AFC,11,5,0,0.688,7-4-0,0.636,4-1-0,0.8,7-1-0,4-4-0
2,New York,Jets,New York Jets,AFC East,AFC,6,10,0,0.375,3-8-0,0.273,1-4-0,0.2,4-5-0,2-5-0
3,New England,Patriots,New England Patriots,AFC East,AFC,4,12,0,0.250,4-7-0,0.364,2-3-0,0.4,1-7-0,3-5-0
4,Kansas City,Chiefs,Kansas City Chiefs,AFC West,AFC,10,6,0,0.625,8-3-0,0.727,3-2-0,0.6,5-4-0,5-2-0
5,Las Vegas,Raiders,Las Vegas Raiders,AFC West,AFC,7,9,0,0.438,5-6-0,0.455,3-2-0,0.6,5-3-0,2-6-0
6,Denver,Broncos,Denver Broncos,AFC West,AFC,8,8,0,0.500,5-6-0,0.455,3-2-0,0.6,5-4-0,3-4-0
7,Los Angeles,Chargers,Los Angeles Chargers,AFC West,AFC,5,11,0,0.312,3-8-0,0.273,1-4-0,0.2,2-6-0,3-5-0
8,Baltimore,Ravens,Baltimore Ravens,AFC North,AFC,13,3,0,0.812,8-3-0,0.727,3-2-0,0.6,6-2-0,7-1-0
9,Cleveland,Browns,Cleveland Browns,AFC North,AFC,11,5,0,0.688,8-3-0,0.727,3-2-0,0.6,8-1-0,3-4-0


In [6]:
for division in teams_df['Division'].unique():
    teams_df[teams_df['Division'] == division].sort_values(by='Pct', ascending=False)